We are working together on this via vscode live share and are talking over zoom. 
In terms of sharing the file we have a github repository set up. We met three times to work in total.


In [1]:
import pandas as pd
import bs4
from bs4 import BeautifulSoup
import requests
from datetime import datetime as dt
import numpy as np
import re

In [2]:
#2
url = "https://www.spaceweatherlive.com/en/solar-activity/top-50-solar-flares"
webpage = requests.get(url)
print(webpage) #response 403, this means that the webserver refused to authorize the request
#to fix do this 
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
webpage = requests.get(url, headers=headers)
print(webpage) #now response 200
soup_content = BeautifulSoup(webpage.content,'html.parser')
pretty = soup_content.prettify()
#print(pretty)
table_html = soup_content.find("table",{"class":"table table-striped"})#['data-value'] #from stackoverflow
df = pd.read_html(table_html.prettify())[0]
df.rename(columns={'Unnamed: 0':"rank",'Unnamed: 1':"x_class",'Unnamed: 2':"date",'Start':"start_time",'Maximum':"max_time",'End':"end_time",'Unnamed: 7':"movie",'Region':"region"},inplace=True)
df.head()


<Response [403]>
<Response [200]>


,rank,x_class,date,region,start_time,max_time,end_time,movie
0,1,X28+,2003/11/04,486,19:29,19:53,20:06,Movie View archive
1,2,X20+,2001/04/02,9393,21:32,21:51,22:03,Movie View archive
2,3,X17.2+,2003/10/28,486,09:51,11:10,11:24,Movie View archive
3,4,X17+,2005/09/07,808,17:17,17:40,18:03,Movie View archive
4,5,X14.4,2001/04/15,9415,13:19,13:50,13:55,Movie View archive


In [3]:
dataFrame = df.drop("movie",axis=1)
dataFrame.head()

for row in dataFrame.iterrows():
    date = pd.to_datetime(row[1].date)
    time_start = pd.to_datetime(row[1].start_time)
    str_time = str(date)[:11]+str(time_start)[11:]
    startTime = pd.to_datetime(str_time)
    dataFrame.at[row[0],'start_time'] = startTime
    
    time_max= pd.to_datetime(row[1].max_time)
    str_time = str(date)[:11]+str(time_max)[11:]
    maxTime = pd.to_datetime(str_time)
    dataFrame.at[row[0],'max_time'] = maxTime
    
    time_end = pd.to_datetime(row[1].end_time)
    str_time = str(date)[:11]+str(time_end)[11:]
    endTime = pd.to_datetime(str_time)
    dataFrame.at[row[0],'end_time'] = endTime
    #date_time_obj = dt.strftime(str_time, '%y-%m-%d %H:%M:%S')
    #dt.combine(date,time_start)
dataFrame = dataFrame.replace('-',np.nan)
dataFrame_update = dataFrame.drop("date",axis=1)
dataFrame_update.rename(columns={'start_time':'start_datetime','max_time':'max_datetime','end_time':'end_datetime'},inplace=True)
region_column = dataFrame_update.region
dataFrame_update=dataFrame_update.drop(['region'],axis=1)
dataFrame_update['region'] = region_column
dataFrame_update.start_datetime = dataFrame_update.start_datetime.astype('datetime64[ns]')
dataFrame_update.max_datetime = dataFrame_update.max_datetime.astype('datetime64[ns]')
dataFrame_update.end_datetime = dataFrame_update.end_datetime.astype('datetime64[ns]')
print(dataFrame_update.dtypes)
dataFrame_update



rank                       int64
x_class                   object
start_datetime    datetime64[ns]
max_datetime      datetime64[ns]
end_datetime      datetime64[ns]
region                     int64
dtype: object


,rank,x_class,start_datetime,max_datetime,end_datetime,region
0,1,X28+,2003-11-04 19:29:00,2003-11-04 19:53:00,2003-11-04 20:06:00,486
1,2,X20+,2001-04-02 21:32:00,2001-04-02 21:51:00,2001-04-02 22:03:00,9393
2,3,X17.2+,2003-10-28 09:51:00,2003-10-28 11:10:00,2003-10-28 11:24:00,486
3,4,X17+,2005-09-07 17:17:00,2005-09-07 17:40:00,2005-09-07 18:03:00,808
4,5,X14.4,2001-04-15 13:19:00,2001-04-15 13:50:00,2001-04-15 13:55:00,9415
5,6,X10,2003-10-29 20:37:00,2003-10-29 20:49:00,2003-10-29 21:01:00,486
6,7,X9.4,1997-11-06 11:49:00,1997-11-06 11:55:00,1997-11-06 12:01:00,8100
7,8,X9.3,2017-09-06 11:53:00,2017-09-06 12:02:00,2017-09-06 12:10:00,2673
8,9,X9,2006-12-05 10:18:00,2006-12-05 10:35:00,2006-12-05 10:45:00,930
9,10,X8.3,2003-11-02 17:03:00,2003-11-02 17:25:00,2003-11-02 17:39:00,486


In [4]:
#Step 3
url = "http://cdaw.gsfc.nasa.gov/CME_list/radio/waves_type2.html"
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
webpage = requests.get(url, headers=headers)
soup_content = BeautifulSoup(webpage.content,'html.parser')
#pretty = soup_content.prettify()
#print(pretty)
row_split = soup_content.find("pre").prettify().split('\n') 


In [5]:
colomns = []
data = []
regex = r'>(.*)<'
for row in row_split[12:-2]:
    row = row.replace('<a','')
    data_split = row.split(' ')#remove ''
    while True:
        try:
            data_split.remove('')
        except:
            break
    try:
        cme_date = re.search( regex ,data_split[9]).groups()[0] #cme
    except:
        cme_date = data_split[9]
    cme_time = data_split[10]
    cpa = data_split[11]
    end_date = data_split[2]
    try:
        end_frequency = re.search( regex ,data_split[5]).groups()[0]
    except:
        end_frequency = data_split[5]
    end_time = data_split[3]
    flare_location = data_split[6]
    flare_region = data_split[7]
    importance = data_split[8]
    try:
        speed =  re.search( regex ,data_split[13]).groups()[0]
    except:
        speed =  data_split[13]
    start_date = data_split[0]
    try:
        start_frequency = re.search( regex ,data_split[4]).groups()[0]
    except:
        start_frequency = data_split[4]
    start_time = data_split[1]
    width = data_split[12]
    data.append([cme_date,cme_time,cpa,end_date,end_frequency,end_time,
                flare_location,flare_region,importance,speed,start_date,
                start_frequency,start_time,width])
columns = ["cme_date","cme_time","cpa","end_date","end_frequency","end_time",
                "flare_location","flare_region","importance","speed","start_date",
                "start_frequency","start_time","width"]
nasa_df = pd.DataFrame(data,columns = columns)
nasa_df.head()

,cme_date,cme_time,cpa,end_date,end_frequency,end_time,flare_location,flare_region,importance,speed,start_date,start_frequency,start_time,width
0,04/01,15:18,74,04/01,4000,14:15,S25E16,8026,M1.3,312,1997/04/01,8000,14:00,79
1,04/07,14:27,Halo,04/07,1000,17:30,S28E19,8027,C6.8,878,1997/04/07,11000,14:30,360
2,05/12,05:30,Halo,05/14,80,16:00,N21W08,8038,C1.3,464,1997/05/12,12000,05:15,360
3,05/21,21:00,263,05/21,500,22:00,N05W12,8040,M1.3,296,1997/05/21,5000,20:20,165
4,09/23,22:02,133,09/23,2000,22:16,S29E25,8088,C1.4,712,1997/09/23,6000,21:53,155


In [6]:
#step 4
nasa_df.replace(['--/--'],[np.nan],inplace=True)
nasa_df.replace(['-----'],[np.nan],inplace=True)
nasa_df.replace(['----'],[np.nan],inplace=True)
nasa_df.replace(['????'],[np.nan],inplace=True)
nasa_df.replace(['--:--'],[np.nan],inplace=True)
nasa_df.replace(['------'],[np.nan],inplace=True)
nasa_df['flare_location'].replace(['BACK'],['Back'],inplace=True)
nasa_df['flare_location'].replace(['back'],['Back'],inplace=True)
nasa_df['flare_region'].replace(['DSF'],['FILA'],inplace=True)
nasa_df['width'].replace(['360h'],['360'],inplace=True)
nasa_df["width"].replace(['---'],[np.nan],inplace=True)
nasa_df.replace(['24:00'], ['23:59'], inplace=True)
halo_flare = []
width_lower_bound = []
helper = nasa_df.iterrows()
start_datetime = []
end_datetime = []
cme_datetime = []
for i in helper:
    #print(i[1]['cpa'])
    if i[1]['cpa'] == "Halo":
        halo_flare.append(True)
    else:
        halo_flare.append(False)
    if '&gt;' in str(i[1]['width']):
        width_lower_bound.append(True)
        nasa_df['width'][i[0]] = i[1]['width'][4:]
    else:
        width_lower_bound.append(False)
    #date time
    
    start_datetime.append(pd.to_datetime(str(i[1]['start_date'])+" "+str(i[1]['start_time'])))
    end_datetime.append(pd.to_datetime((str(i[1]['start_date'])[0:5]+str(i[1]['end_date'])+" "+str(i[1]['end_time']))))
    try:
        cme_datetime.append(pd.to_datetime((str(i[1]['start_date'])[0:5]+str(i[1]['cme_date'])+" "+str(i[1]['cme_time']))))
    except:
        cme_datetime.append(np.datetime64("NaT"))
nasa_df.drop(columns = ['start_time', 'start_date', 'end_date', 'end_time', 'cme_date', 'cme_time'], inplace= True)
nasa_df['is_flare'] = halo_flare
nasa_df['width_lower_bound'] = width_lower_bound
nasa_df['start_datetime'] = start_datetime
nasa_df['end_datetime'] = end_datetime
nasa_df['cme_datetime'] = cme_datetime
nasa_df['cpa'].replace(['Halo'],[np.nan],inplace=True) #do not run more than once

In [7]:

nasa_df = nasa_df.astype({'cpa':'float','end_frequency':'float','speed':'float','start_frequency':'float','width':'float'})
print(nasa_df.dtypes)
nasa_df= nasa_df[['start_datetime','end_datetime','start_frequency','end_frequency','flare_location','flare_region','importance', 'cme_datetime','cpa', 'width','speed','is_flare','width_lower_bound']]
nasa_df.head(10)

cpa                         float64
end_frequency               float64
flare_location               object
flare_region                 object
importance                   object
speed                       float64
start_frequency             float64
width                       float64
is_flare                       bool
width_lower_bound              bool
start_datetime       datetime64[ns]
end_datetime         datetime64[ns]
cme_datetime         datetime64[ns]
dtype: object


,start_datetime,end_datetime,start_frequency,end_frequency,flare_location,flare_region,importance,cme_datetime,cpa,width,speed,is_flare,width_lower_bound
0,1997-04-01 14:00:00,1997-04-01 14:15:00,8000.0,4000.0,S25E16,8026,M1.3,1997-04-01 15:18:00,74.0,79.0,312.0,False,False
1,1997-04-07 14:30:00,1997-04-07 17:30:00,11000.0,1000.0,S28E19,8027,C6.8,1997-04-07 14:27:00,NaN,360.0,878.0,True,False
2,1997-05-12 05:15:00,1997-05-14 16:00:00,12000.0,80.0,N21W08,8038,C1.3,1997-05-12 05:30:00,NaN,360.0,464.0,True,False
3,1997-05-21 20:20:00,1997-05-21 22:00:00,5000.0,500.0,N05W12,8040,M1.3,1997-05-21 21:00:00,263.0,165.0,296.0,False,False
4,1997-09-23 21:53:00,1997-09-23 22:16:00,6000.0,2000.0,S29E25,8088,C1.4,1997-09-23 22:02:00,133.0,155.0,712.0,False,False
5,1997-11-03 05:15:00,1997-11-03 12:00:00,14000.0,250.0,S20W13,8100,C8.6,1997-11-03 05:28:00,240.0,109.0,227.0,False,False
6,1997-11-03 10:30:00,1997-11-03 11:30:00,14000.0,5000.0,S16W21,8100,M4.2,1997-11-03 11:11:00,233.0,122.0,352.0,False,False
7,1997-11-04 06:00:00,1997-11-05 04:30:00,14000.0,100.0,S14W33,8100,X2.1,1997-11-04 06:10:00,NaN,360.0,785.0,True,False
8,1997-11-06 12:20:00,1997-11-07 08:30:00,14000.0,100.0,S18W63,8100,X9.4,1997-11-06 12:10:00,NaN,360.0,1556.0,True,False
9,1997-11-27 13:30:00,1997-11-27 14:00:00,14000.0,7000.0,N17E63,8113,X2.6,1997-11-27 13:56:00,98.0,91.0,441.0,False,False


In [70]:
#Part 2 start
#def removeX
nasa_x_df = nasa_df[nasa_df['importance'].str[0]=='X']
nasa_x_df["importance"] = nasa_x_df["importance"].str[1:]
nasa_x_df = nasa_x_df.astype({'importance':'float'}).sort_values(by='importance',ascending = False)
nasa_x_df = nasa_x_df.astype({'importance':'string'})
nasa_x_df["importance"] = 'X' + nasa_x_df["importance"]
compare_df = nasa_x_df.head(50)
display(compare_df.head())
dataFrame_update.head()
#We were able to replicate the top 50 solar flares pretty well, but when comparing
#the two dataframes, some importance classification did not line up perfectly.


/tmp/ipykernel_547/1742918134.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nasa_x_df["importance"] = nasa_x_df["importance"].str[1:]


,start_datetime,end_datetime,start_frequency,end_frequency,flare_location,flare_region,importance,cme_datetime,cpa,width,speed,is_flare,width_lower_bound
240,2003-11-04 20:00:00,2003-11-04 23:59:00,10000.0,200.0,S19W83,10486,X28.0,2003-11-04 19:54:00,NaN,360.0,2657.0,True,False
117,2001-04-02 22:05:00,2001-04-03 02:30:00,14000.0,250.0,N19W72,9393,X20.0,2001-04-02 22:06:00,261.0,244.0,2505.0,False,False
233,2003-10-28 11:10:00,2003-10-29 23:59:00,14000.0,40.0,S16E08,10486,X17.0,2003-10-28 11:30:00,NaN,360.0,2459.0,True,False
126,2001-04-15 14:05:00,2001-04-16 13:00:00,14000.0,40.0,S20W85,9415,X14.0,2001-04-15 14:06:00,245.0,167.0,1199.0,False,False
234,2003-10-29 20:55:00,2003-10-29 23:59:00,11000.0,500.0,S15W02,10486,X10.0,2003-10-29 20:54:00,NaN,360.0,2029.0,True,False


,rank,x_class,start_datetime,max_datetime,end_datetime,region
0,1,X28+,2003-11-04 19:29:00,2003-11-04 19:53:00,2003-11-04 20:06:00,486
1,2,X20+,2001-04-02 21:32:00,2001-04-02 21:51:00,2001-04-02 22:03:00,9393
2,3,X17.2+,2003-10-28 09:51:00,2003-10-28 11:10:00,2003-10-28 11:24:00,486
3,4,X17+,2005-09-07 17:17:00,2005-09-07 17:40:00,2005-09-07 18:03:00,808
4,5,X14.4,2001-04-15 13:19:00,2001-04-15 13:50:00,2001-04-15 13:55:00,9415


In [100]:
#Question 2
def date_compare(date):
    print(date)
    date_split = date.split(" ")
    if int(date_split[0])>=1:
        return False
    new_split = date_split[2].split(':')
    if int(new_split[0])>2:
        return False
    return True
        
def comparer(df1,df2):
    val = 0
    #1 of top 50 == > {1:10,2:7,:...:50..7}
    pass
    #start_DT ==> within 2 hours of eachother then val +=2
    #X_class ==> within 1.5 of eachother then val +=2
    #region => contain 3 nums in correct order of eachother then val +=2, if 2 then val+=1
    #end_DT ==> within 2 hours of eachother then val +=2
    
    start_diff = abs(df1['start_datetime']-df2['start_datetime'])
    end_diff = abs(df1['end_datetime']- df2['end_datetime'])
    #cme_diff = abs(df1['end_datetime']- df2['cme_datetime'])
    if date_compare(str(start_diff)):
        val+=2
    if date_compare(str(end_diff)):
        val+=2
    return val
#print(compare_df.flare_region.unique())
#dataFrame_update.region.unique()
arr_ranking_temp = []
for top_i in dataFrame_update.iterrows():
    #0=id,1 =info
    temp_dict = dict()
    for nasa_i in compare_df.iterrows():
        temp_dict[nasa_i[0]] = comparer(top_i[1],nasa_i[1])
    arr_ranking_temp.append(temp_dict)
#arr_ranking_temp
#arr_ranking = []
#for i in arr_ranking_temp:
#    arr_ranking.append(max(i,key=i.get))
#arr_ranking


0 days 00:31:00
0 days 03:53:00
945 days 21:24:00
945 days 17:36:00
7 days 08:19:00
5 days 20:07:00
933 days 05:24:00
932 days 07:06:00
5 days 22:34:00
5 days 20:07:00
2189 days 07:09:00
2188 days 11:36:00
5054 days 16:36:00
5055 days 11:54:00
1126 days 15:21:00
1126 days 23:54:00
2 days 01:59:00
1 days 19:06:00
5058 days 20:33:00
5059 days 10:44:00
442 days 11:46:00
442 days 20:24:00
2834 days 12:51:00
2834 days 12:29:00
1127 days 23:31:00
1130 days 03:53:00
675 days 00:16:00
675 days 01:54:00
1208 days 08:59:00
1207 days 05:36:00
941 days 23:54:00
941 days 18:16:00
3045 days 05:31:00
3046 days 22:54:00
801 days 02:39:00
800 days 21:06:00
3765 days 05:27:00
3765 days 15:22:00
469 days 18:39:00
469 days 16:06:00
1073 days 02:29:00
1073 days 02:51:00
1 days 09:29:00
1 days 07:36:00
439 days 14:31:00
439 days 14:29:00
160 days 18:29:00
159 days 19:36:00
1134 days 07:16:00
1134 days 14:34:00
675 days 22:54:00
675 days 17:06:00
471 days 21:59:00
471 days 21:46:00
3478 days 05:47:00
3478 da

2189 days 08:11:00
2189 days 11:58:00
1243 days 10:16:00
1243 days 14:29:00
2181 days 23:21:00
2183 days 11:58:00
1256 days 02:16:00
1257 days 00:59:00
2183 days 09:06:00
2183 days 11:58:00
0 days 00:31:00
0 days 20:29:00
7244 days 00:16:00
7244 days 19:59:00
3315 days 23:01:00
3316 days 07:59:00
2187 days 05:41:00
2187 days 12:59:00
7248 days 04:13:00
7248 days 18:49:00
2631 days 19:26:00
2632 days 04:29:00
5023 days 20:31:00
5023 days 20:34:00
3317 days 07:11:00
3319 days 11:58:00
2864 days 07:56:00
2864 days 09:59:00
980 days 22:41:00
982 days 02:29:00
1247 days 07:46:00
1247 days 13:49:00
5234 days 13:11:00
5236 days 06:59:00
1388 days 05:01:00
1388 days 10:59:00
5954 days 13:07:00
5954 days 23:27:00
1719 days 13:01:00
1719 days 15:59:00
1116 days 05:11:00
1116 days 05:14:00
2187 days 22:11:00
2188 days 00:29:00
2628 days 22:11:00
2628 days 22:34:00
2028 days 13:11:00
2029 days 12:29:00
3323 days 14:56:00
3323 days 22:39:00
1513 days 08:46:00
1513 days 14:59:00
1717 days 09:41:00
1

5022 days 23:38:00
2220 days 04:17:00
2220 days 23:52:00
1707 days 20:58:00
1707 days 12:08:00
2836 days 14:18:00
2836 days 07:08:00
2224 days 08:14:00
2224 days 22:42:00
2392 days 00:33:00
2391 days 15:38:00
0 days 00:32:00
0 days 00:27:00
1706 days 12:48:00
1704 days 08:09:00
2159 days 12:03:00
2159 days 10:08:00
4042 days 21:18:00
4041 days 17:38:00
3776 days 12:13:00
3776 days 06:18:00
210 days 17:12:00
212 days 10:52:00
3635 days 14:58:00
3635 days 09:08:00
930 days 17:08:00
931 days 03:20:00
3304 days 06:58:00
3304 days 04:08:00
3907 days 14:48:00
3907 days 14:53:00
2835 days 21:48:00
2835 days 19:38:00
2394 days 21:48:00
2394 days 21:33:00
2995 days 06:48:00
2994 days 07:38:00
1700 days 05:03:00
1699 days 21:28:00
3510 days 11:13:00
3510 days 05:08:00
3306 days 10:18:00
3306 days 09:48:00
643 days 17:28:00
644 days 00:12:00
3272 days 06:03:00
3272 days 04:43:00
643 days 08:27:00
643 days 11:02:00
1365 days 14:36:00
1365 days 15:06:00
4842 days 23:23:00
4842 days 23:33:00
2836 da

4381 days 10:43:00
452 days 13:58:00
452 days 22:43:00
676 days 03:22:00
675 days 20:17:00
4384 days 19:10:00
4385 days 09:33:00
231 days 13:37:00
231 days 04:47:00
2160 days 11:28:00
2160 days 11:18:00
453 days 22:08:00
456 days 02:42:00
0 days 22:53:00
1 days 00:43:00
1882 days 10:22:00
1881 days 06:47:00
1616 days 01:17:00
1615 days 19:27:00
2371 days 04:08:00
2372 days 21:43:00
1475 days 04:02:00
1474 days 22:17:00
3091 days 04:04:00
3091 days 14:11:00
1143 days 20:02:00
1143 days 17:17:00
1747 days 03:52:00
1747 days 04:02:00
675 days 10:52:00
675 days 08:47:00
234 days 10:52:00
234 days 10:42:00
834 days 19:52:00
833 days 20:47:00
460 days 05:53:00
460 days 13:23:00
1350 days 00:17:00
1349 days 18:17:00
1145 days 23:22:00
1145 days 22:57:00
2804 days 04:24:00
2804 days 11:03:00
1111 days 19:07:00
1111 days 17:52:00
2803 days 19:23:00
2803 days 21:53:00
3526 days 01:32:00
3526 days 01:57:00
2682 days 12:27:00
2682 days 12:42:00
675 days 19:37:00
675 days 19:52:00
235 days 21:52:00

1538 days 15:53:00
1538 days 16:12:00
468 days 00:57:00
468 days 00:38:00
907 days 22:42:00
908 days 23:13:00
301 days 13:33:00
300 days 04:47:00
1698 days 10:48:00
1698 days 10:47:00
841 days 02:07:00
841 days 02:53:00
468 days 18:54:00
468 days 00:48:00
605 days 08:53:00
605 days 02:47:00
776 days 08:58:00
774 days 15:47:00
3129 days 01:52:00
3129 days 06:13:00
1145 days 21:27:00
1146 days 00:13:00
3332 days 22:12:00
3333 days 14:53:00
4112 days 14:50:00
4112 days 21:45:00
1721 days 18:18:00
1720 days 20:17:00
605 days 19:08:00
605 days 09:47:00
466 days 13:58:00
466 days 14:07:00
838 days 16:07:00
839 days 19:13:00
909 days 09:07:00
909 days 15:13:00
604 days 05:18:00
604 days 05:12:00
1073 days 03:26:00
1073 days 07:03:00
127 days 05:31:00
127 days 09:34:00
1065 days 18:36:00
1067 days 07:03:00
139 days 21:31:00
140 days 20:04:00
1067 days 04:21:00
1067 days 07:03:00
1116 days 04:14:00
1115 days 08:26:00
6127 days 19:31:00
6128 days 15:04:00
2199 days 18:16:00
2200 days 03:04:00
10

2683 days 01:17:00
2683 days 01:33:00
676 days 08:27:00
676 days 08:43:00
236 days 10:42:00
235 days 10:08:00
1445 days 22:57:00
1444 days 14:08:00
2842 days 20:12:00
2842 days 20:08:00
303 days 07:17:00
303 days 06:28:00
1613 days 04:18:00
1612 days 10:09:00
1749 days 18:17:00
1749 days 12:08:00
1920 days 18:22:00
1919 days 01:08:00
1984 days 16:28:00
1984 days 20:52:00
1 days 12:03:00
1 days 14:52:00
2188 days 12:48:00
2189 days 05:32:00
2968 days 05:26:00
2968 days 12:24:00
2866 days 03:42:00
2865 days 05:38:00
1750 days 04:32:00
1749 days 19:08:00
1610 days 23:22:00
1610 days 23:28:00
305 days 17:17:00
304 days 14:08:00
235 days 00:17:00
234 days 18:08:00
1748 days 14:42:00
1748 days 14:33:00
254 days 17:49:00
254 days 14:02:00
1200 days 15:44:00
1200 days 11:31:00
262 days 02:39:00
260 days 14:02:00
1187 days 23:44:00
1187 days 01:01:00
260 days 16:54:00
260 days 14:02:00
2444 days 01:29:00
2443 days 05:31:00
4799 days 22:16:00
4800 days 17:59:00
871 days 21:01:00
872 days 05:59:0

841 days 22:06:00
911 days 12:21:00
911 days 18:06:00
602 days 02:04:00
602 days 02:19:00
1802 days 15:06:00
1802 days 17:46:00
856 days 17:11:00
856 days 20:17:00
1795 days 06:16:00
1796 days 17:46:00
869 days 09:11:00
870 days 06:47:00
1796 days 16:01:00
1796 days 17:46:00
386 days 16:34:00
385 days 21:43:00
6857 days 07:11:00
6858 days 01:47:00
2929 days 05:56:00
2929 days 13:47:00
1800 days 12:36:00
1800 days 18:47:00
6861 days 11:08:00
6862 days 00:37:00
2245 days 02:21:00
2245 days 10:17:00
4637 days 03:26:00
4637 days 02:22:00
2930 days 14:06:00
2932 days 17:46:00
2477 days 14:51:00
2477 days 15:47:00
594 days 05:36:00
595 days 08:17:00
860 days 14:41:00
860 days 19:37:00
4847 days 20:06:00
4849 days 12:47:00
1001 days 11:56:00
1001 days 16:47:00
5567 days 20:02:00
5568 days 05:15:00
1332 days 19:56:00
1332 days 21:47:00
729 days 12:06:00
729 days 11:02:00
1801 days 05:06:00
1801 days 06:17:00
2242 days 05:06:00
2242 days 04:22:00
1641 days 20:06:00
1642 days 18:17:00
2936 days 

5748 days 04:07:00
5748 days 23:46:00
1820 days 02:52:00
1820 days 11:46:00
691 days 09:32:00
691 days 16:46:00
5752 days 08:04:00
5752 days 22:36:00
1135 days 23:17:00
1136 days 08:16:00
3528 days 00:22:00
3528 days 00:21:00
1821 days 11:02:00
1823 days 15:45:00
1368 days 11:47:00
1368 days 13:46:00
514 days 21:28:00
513 days 17:44:00
248 days 12:23:00
248 days 06:24:00
3738 days 17:02:00
3740 days 10:46:00
107 days 15:08:00
107 days 09:14:00
4458 days 16:58:00
4459 days 03:14:00
223 days 16:52:00
223 days 19:46:00
379 days 14:58:00
379 days 14:59:00
692 days 02:02:00
692 days 04:16:00
1133 days 02:02:00
1133 days 02:21:00
532 days 17:02:00
533 days 16:16:00
1827 days 18:47:00
1828 days 02:26:00
17 days 12:37:00
17 days 18:46:00
221 days 13:32:00
221 days 14:06:00
4171 days 17:18:00
4172 days 00:06:00
255 days 17:47:00
255 days 19:11:00
4171 days 08:17:00
4171 days 10:56:00
4893 days 14:26:00
4893 days 15:00:00
1314 days 23:33:00
1314 days 23:39:00
691 days 17:17:00
691 days 17:11:00


611 days 13:21:00
1905 days 17:13:00
1905 days 23:31:00
95 days 11:03:00
95 days 15:51:00
299 days 11:58:00
299 days 11:11:00
4249 days 15:44:00
4249 days 21:11:00
333 days 16:13:00
333 days 16:16:00
4249 days 06:43:00
4249 days 08:01:00
4971 days 12:52:00
4971 days 12:05:00
1237 days 01:07:00
1237 days 02:34:00
769 days 15:43:00
769 days 14:16:00
1209 days 13:28:00
1210 days 12:51:00
0 days 01:13:00
1 days 08:51:00
1396 days 20:02:00
1396 days 21:09:00
1142 days 16:53:00
1142 days 16:31:00
167 days 04:08:00
166 days 11:10:00
303 days 18:07:00
303 days 13:09:00
474 days 18:12:00
473 days 02:09:00
3430 days 16:38:00
3430 days 19:51:00
1447 days 12:13:00
1447 days 13:51:00
3634 days 12:58:00
3635 days 04:31:00
4414 days 05:36:00
4414 days 11:23:00
1420 days 03:32:00
1419 days 06:39:00
304 days 04:22:00
303 days 20:09:00
164 days 23:12:00
165 days 00:29:00
1140 days 06:53:00
1141 days 08:51:00
1210 days 23:53:00
1211 days 04:51:00
302 days 14:32:00
302 days 15:34:00
